# Persistence Diagrams Demonstration

We demonstrate how to use various Python packages to compute and display persistence diagrams. We use the standard Python packages matplotlib, numpy, sklearn as well as the specialized (for TDA) packages ripser and persim. Ripser and persim need to be pip installed before running this notebook.

This is a fleshed out version of the demonstration available at https://github.com/scikit-tda/ripser.py

In [ ]:
# Import packages
from ripser import ripser
# ripser is a package for computing barcodes from Vietoris-Rips complexes
from persim import plot_diagrams 
# persim is a package for displaying and computing distances between persistence diagrams
import matplotlib.pyplot as pl
from mpl_toolkits.mplot3d import Axes3D
# The above are for producing scatterplots of our data
import numpy as np
# This is a standard package for linear algebra and statistics

We begin by defining a toy dataset. Let's just take a bunch of random points in the plane.

In [ ]:
data = np.random.random((100,2)) # Define a random point cloud of 100 points in the plane
fig = pl.figure() # Create a figure to display the random point cloud
ax1 = fig.add_subplot(121)
ax1.plot(data[:, 0], data[:, 1],'ob') # Plot the data on the axes. +b plots as a scatter plot of blue + signs.
ax1.axis('equal'); # Set the aspect ratio to use equal axis scales.

Now we compute the persistent homology of the Vietoris-Rips complex of the dataset. This is done by applying the ripser function. Ripser has lots of options, such as range of dimensions to compute, type of data etc. If we don't specify anything, then it will run the computation assuming the data is a pointcloud (which it is, in our case).

In [ ]:
ripserData = ripser(data)

Let's take a look at the output of ripser.

In [ ]:
ripserData

The types of things that ripser computed are listed under various 'keys'. Let's take a look at those.

In [ ]:
ripserData.keys()

The computation that we really care about is under the 'dgms' key.

In [ ]:
diagrams = ripser(data)['dgms'] # Pick off the dgms part of the ripser output
print(diagrams) # Look at the the output

We see that the output of diagrams is a pair of arrays. These are points in the persistence diagram for degree-0 and degree-1 persistent homology, respectively. We can now plot these diagrams. This can be done on separate axes, or on the same axis.

In [ ]:
plot_diagrams(diagrams[0]) # Just degree-0

In [ ]:
plot_diagrams(diagrams[1]) # Just degree-1

In [ ]:
plot_diagrams(diagrams) # Both degree-0 and degree-1

Let's look at some other examples. First we define a function to randomly sample from Euclidean spheres. Then we sample 250 points from a sphere in 3-space and plot the result.

In [ ]:
def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(ndim, npoints)
    vec /= np.linalg.norm(vec, axis=0)
    vec = np.transpose(vec)
    return vec

sphere250 = sample_spherical(250)

fig = pl.figure()
ax = Axes3D(fig)
ax.scatter(sphere250[:,0],sphere250[:,1],sphere250[:,2], c='b', marker='o');
ax.set_aspect('equal');

Now let's compute the persistence diagrams for the data. We can specify that we want to compute homology up to degree-2. We then plot the persistence diagrams on the same axes. I'm interested in how long these computations take, so I will also run a timer.

In [ ]:
import time

In [ ]:
start0 = time.time()
diagrams = ripser(sphere250,maxdim=2)['dgms']
plot_diagrams(diagrams)
end0 = time.time()

print('Computation Time: ' + str(end0 - start0) + ' seconds')

Observe in the persistence diagrams that there is one highly persistent 0-cycle and a single 2-cycle. These reflect the topology of the sphere. Since 250 points sparsely covers the sphere, we see there are lots of spurious loops which form and die out quickly in the Vietoris-Rips complex. Let's see what happens if we sample the sphere more densely.

In [ ]:
sphere500 = sample_spherical(500)

fig = pl.figure()
ax = Axes3D(fig)
ax.scatter(sphere500[:,0],sphere500[:,1],sphere500[:,2], c='b', marker='o');
ax.set_aspect('equal');

In [ ]:
start0 = time.time()
diagrams = ripser(sphere500,maxdim=2)['dgms']
plot_diagrams(diagrams, show=True)
end0 = time.time()

print('Computation Time: ' + str(end0 - start0) + ' seconds')

The above diagrams show that the more densely sampled sphere has much more pronounced persistent features. Now let's look at "noisy" samples from a circle.

In [ ]:
data = sample_spherical(100,ndim=2)+.5*np.random.random((100,2))
# Generate the noisy circle.

# Plot the data as a scatter plot.
fig = pl.figure()
ax1 = fig.add_subplot(121)
ax1.plot(data[:, 0], data[:, 1], 'ob', label='Source samples');
ax1.axis('equal');

In [ ]:
dgms = ripser(data)['dgms']
plot_diagrams(dgms, show=True)

Playing with parameters, we can make the circle more densely sampled or much noisier.

In [ ]:
numsamp = 50
noiseLevel = .5

data = sample_spherical(numsamp,ndim=2)+noiseLevel*np.random.random((numsamp,2))

fig = pl.figure()
ax1 = fig.add_subplot(121)
ax1.plot(data[:, 0], data[:, 1], 'ob');
ax1.axis('equal')
dgms = ripser(data)['dgms']
fig2 = pl.figure()
plot_diagrams(dgms, show=True)

By default, ripser computes persistent homology over the integer ring (which we haven't covered in class). We can force it to compute over the field with two elements (also called Z/2Z) using the following commands.

In [ ]:
# Homology over Z/2Z
dgms = ripser(data, coeff=2)['dgms']
plot_diagrams(dgms, plot_only=[1], title="Homology over Z/2Z", show=True)

Another option in Ripser is to display the persistance diagram tilted so that the x=y line becomes the x-axis. This is called a "Lifetime" persistence diagram.

In [ ]:
plot_diagrams(dgms, lifetime=True)